# How to use nodejs simple-imap?

How to log in to Gmail using imap-simple/node IMAP?

How to temporarily allow less secure apps?

https://myaccount.google.com/lesssecureapps

TODO: convert to selenium/sikuli script



How to connect to Gmail using simple-imap?

How do I connect using simple-imap and {credentials}?

node imap client?



In [ ]:
var fs = require('fs');
var path = require('path');
var imaps = require('imap-simple');
var mime = require('mime');
var importer = require('../Core');
var getCredentials = importer.import('decrypt passwords');

var output = path.join(process.cwd(), 'output');
var credentials = getCredentials('accounts.google.com');
process.env.GOOGLE_USER = credentials.Email;
process.env.GOOGLE_PASS = credentials.Passwd;

var config = {
    imap: {
        user: process.env.GOOGLE_USER,
        password: process.env.GOOGLE_PASS,
        host: 'imap.gmail.com',
        port: 993,
        tls: true,
        autotls: 'required',
        authTimeout: 3000
    }
};

function imapClient() {
    var connection;
    return imaps.connect(config)
        .then((conn) => {
            connection = conn;
            return conn.openBox('INBOX');
        })
        .then(() => connection)
}
module.exports = imapClient;


How to search for messages using simple-imap?

How to search for messages in the last few {days}?

How to search for messages {from} a sender's email address?


In [ ]:
var chrono = require('chrono-node');
var importer = require('../Core');
var {
    imapClient,
} = importer.import([
    'node imap client',
]);

function flatten(messages) {
    return messages.map(message => {
        var header = message.parts.filter(function (part) {
            return part.which.indexOf('HEADER') > -1;
        });
        var text = message.parts.filter(function (part) {
            return part.which.indexOf('TEXT') > -1;
        });
        var subject = header[0].body.subject[0];
        var from = header[0].body.from[0];
        var to = header[0].body.to[0];
        var date = header[0].body.date[0];
        from = ((/.+<(.*?)>/ig).exec(from) || [])[1] || from;
        to = ((/.+<(.*?)>/ig).exec(to) || [])[1] || to;
        return {
            id: message.attributes.uid,
            email: from,
            subject,
            body: text[0].body,
            date: chrono.parseDate(date),
            to
        };
    })
}

function searchImap(from = '*', subject = '*', days = 7, mailbox = 'INBOX') {
    // Fetch emails from the last number of {days}
    var delay = 24 * 3600 * 1000 * days;
    var lastWeek = new Date();
    lastWeek.setTime(Date.now() - delay);
    lastWeek = lastWeek.toISOString();
    var searchCriteria = [['SINCE', lastWeek]];
    if(from !== '*') {
        searchCriteria.push(['FROM', from])
    }
    if(subject !== '*') {
        searchCriteria.push(['SUBJECT', subject])
    }
    var fetchOptions = {
        bodies: ['HEADER.FIELDS (FROM TO SUBJECT DATE)', 'TEXT'],
        struct: true
    };
    var connection;
    return imapClient()
        .then(c => {
            connection = c;
            if(mailbox !== 'INBOX') {
                return connection.openBox(mailbox)
            }
        })
        .then(() => connection.search(searchCriteria, fetchOptions))
        .then(r => {
            r.flatten = () => flatten(r);
            return r;
        })
        .catch(e => console.log(e))
}
module.exports = searchImap;


scan commands email?


In [4]:
var importer = require('../Core');
var {
    searchImap,
    sendEmail,
    getOauthClient,
    filterCommand,
    storeResult
} = importer.import([
    'search imap messages',
    'send email',
    'filter command permission',
    'store rpc result'
]);

var options = {
    calendarId: 'Commands'
};

function scanCommandsEmail() {
    return searchImap('*', 'megamind', 1, 'Megamind')
        .then(messages => messages.flatten())
        // get settings for each
        .then(friends => {
            const hasCommands = friends.map(f => Object.assign(f, {
                command: (/megamind\s*(.*)/ig).exec(f.subject)[1].trim()
            })).filter(f => f.command.length > 0
                       // filter out responses from my own INBOX so I can test this service myself
                        && !f.body.match(/^\s* Mm\s*$/gm));
            return importer.runAllPromises(hasCommands.map(f => resolve => {
                return filterCommand(f.command, f.date, f.id, f.email)
                    .then(props => resolve(Object.assign(f, props)));
            }))
        })
        // generate and send responses
        .then(friends => {
            return importer.runAllPromises(friends.map(f => resolve => {
                return storeResult(f)
                    .then(response => sendEmail(
                        f.email,
                        JSON.stringify(response, null, 4) + '\n Mm\n',
                        f.subject))
                    .catch(e => console.log(e))
                    .then(r => resolve(r))
            }))
        })
        .catch(e => console.log(e))
}
module.exports = scanCommandsEmail;

if(typeof $$ !== 'undefined') {
    $$.async();
    scanCommandsEmail()
        .then(r => $$.sendResult(r))
        .catch((e) => $$.sendError(e))
}


{ timeMin: '2017-11-27T00:00:00-00:00',
  timeMax: '2017-11-28T00:00:00-00:00' }
Using calendar: Commands - kqn9q1hpho8q2ti5mqvv6tkth8@group.calendar.google.com


[]

How to list the subjects from simple-imap messages?

How to list the sender's email address from simple-imap messages?


In [ ]:
htmlPrint = '';
messages.forEach(message => {
    var header = message.parts.filter(function (part) {
        return part.which.indexOf('HEADER') > -1;
    });
    var subject = header[0].body.subject[0];
    var from = header[0].body.from[0];
    htmlPrint += '<li>subject: ' + subject + ', from: ' + from + '</li>\n';
});
$$.mime({'text/markdown': 'Usage:\n\n```html\n' + htmlPrint + '\n```\nOuput:\n'});



How to extract a list of image attachments from an email?

How do I extract attachments from {message}

How to get attachments using simple-imap?

How to get attachements from the last few {days}?

How to get messages {from} and email address?



In [ ]:
$$.async();
attachments = []
Promise.all(messages.map(message => {
    return Promise.all(imaps.getParts(message.attributes.struct)
        .filter((part) => part.disposition && part.disposition.type === 'ATTACHMENT')
        // retrieve the attachments only of the messages with attachments 
        .map((part) => connection.getPartData(message, part).then((partData) => ({
            filename: part.disposition.params.filename,
            data: partData
        }))))
        .then(a => attachments = attachments.concat(a));
})).then((o) => $$.done(attachments)).catch(e => $$.done(e));

How to save an attachments from simple-imap?

How to save a {file}?

How to save binary {data}?

How to {output} all attachments in the requested {format}?



In [ ]:
// TODO: convert the attachments to a virtual filesystem, lay it on top of the current app, run end-to-end tests
if (!fs.existsSync(output)) {
    fs.mkdirSync(output);
}
var result = attachments.map((attachment) => {
    return new Promise((resolve, reject) => fs.writeFile(
        path.join(output, attachment.filename),
        attachment.data,
        'binary',
        function (err) {
            if (err) reject(err);
            else resolve(attachment);
        }));
});
$$.async();
Promise.all(result).then(images => {
    html = '';
    htmlPrint = '';
    images.forEach((i) => {
        var filename = i.filename.split('/').pop();
        var ext = mime.lookup(i.filename);
        html += '<img src="data:' + ext + ';base64,' + (new Buffer(i.data, 'binary')).toString('base64') + '" />';
        htmlPrint += '<li><img src="/assets/' + filename + '" /></li>\n';
    });
    $$.mime({'text/markdown': 'Usage:\n\n```html\n' + htmlPrint + '\n```\nOutput:\n' + html});
}).catch(e => $$.done(e));



send email?


In [2]:
var nodemailer = require('nodemailer')
var importer = require('../Core');
var getCredentials = importer.import('decrypt passwords');

var credentials = getCredentials('accounts.google.com');
process.env.GOOGLE_USER = credentials.Email;
process.env.GOOGLE_PASS = credentials.Passwd;

function sendEmail(to, text, subject, from) {
    // create reusable transporter object using the default SMTP transport
    let transporter = nodemailer.createTransport({
        host: 'smtp.gmail.com',
        port: 465,
        secure: true, // true for 465, false for other ports
        auth: {
            user: process.env.GOOGLE_USER, // generated ethereal user
            pass: process.env.GOOGLE_PASS  // generated ethereal password
        }
    });

    // setup email data with unicode symbols
    let mailOptions = {
        from: from, // sender address
        to: to, // list of receivers
        subject: subject, // Subject line
        text: text, // plain text body
        html: text // html body
    };

    // send mail with defined transport object
    return new Promise((resolve, reject) => {
        transporter.sendMail(mailOptions, (error, info) => {
            if (error) {
                return reject(error);
            }
            resolve(info.messageId);
        });
    });
}
module.exports = sendEmail;



[Function: sendEmail]

Append email to drafts


In [ ]:
var imaps = require('imap-simple');
 
var config = {
    imap: {
        user: 'your@email.address',
        password: 'yourpassword',
        host: 'imap.gmail.com',
        port: 993,
        tls: true,
        authTimeout: 3000
    }
};
 
imaps.connect(config).then(function (connection) {
  const message = `Content-Type: text/plain
To: jhannes@gmail.com
Subject: Hello world
 
Hi
This is a test message
`;
  connection.append(message.toString(), {mailbox: 'Drafts', flags: '\\Draft'});
});

